In [47]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [5]:
url = 'https://bardentreffen.nuernberg.de/programm'
page = requests.get(url)
page

<Response [200]>

In [8]:
soup = BeautifulSoup(page.text, 'html.parser')
print (soup)

<!DOCTYPE html>

<html lang="de">
<head>
<meta charset="utf-8"/>
<!-- 
	This website is powered by TYPO3 - inspiring people to share!
	TYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.
	TYPO3 is copyright 1998-2022 of Kasper Skaarhoj. Extensions are copyright of their respective owners.
	Information and contribution at https://typo3.org/
-->
<title>Programm</title>
<meta content="TYPO3 CMS" name="generator">
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/typo3conf/ext/user_bar/Resources/Public/Css/style.css?1628760961" media="all" rel="stylesheet" type="text/css"/>
<link href="/typo3temp/assets/css/e7a43898ce.css?1617970632" media="all" rel="stylesheet" type="text/css"/>
<script src="/typo3conf/ext/user_bar/Resources/Public/JavaScript/jquery-2.1.1.min.js?1628760964"></script>
<script src="/typo3conf/ext/user_bar/Resources/Public/JavaScript/jquery-ui.min.js?1628760964"></script>
<

In [66]:
data = []
for anker in soup.find_all('a'):
    tag = BeautifulSoup(str(anker), 'html.parser')
    stage_time = tag.a.get('data-time')
    location = tag.a.get('data-location')
    title = tag.a.get('data-title')
    if stage_time and location and title:
        data.append([datetime.fromtimestamp(int(stage_time)),
                    location,
                    title])

df = pd.DataFrame(data,
                 columns=['Zeit', 'Location', 'Titel'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Zeit      95 non-null     datetime64[ns]
 1   Location  95 non-null     object        
 2   Titel     95 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 2.4+ KB


In [69]:
df['Tag'] = pd.to_datetime(df['Zeit']).dt.date
df['Uhrzeit'] = pd.to_datetime(df['Zeit']).dt.time
df.head()

,Zeit,Location,Titel,Tag,Uhrzeit
0,2022-07-31 21:20:00,Lorenzer Platz,A Tale Of Golden Keys,2022-07-31,21:20:00
1,2022-07-31 21:15:00,Insel Schütt,Mal Élevé,2022-07-31,21:15:00
2,2022-07-31 21:00:00,Sebalder Platz,Phil Siemers,2022-07-31,21:00:00
3,2022-07-31 21:00:00,St. Katharina,L'Alba,2022-07-31,21:00:00
4,2022-07-31 20:45:00,Trödelmarkt,Csókolom,2022-07-31,20:45:00


In [72]:
#df.set_index(['Location', 'Zeit']).sort_index(axis=0, level=[0, 1])
df_pivoted = df.pivot(index=['Tag', 'Uhrzeit'], columns='Location', values='Titel')
df_pivoted.head()

Location                 Hauptmarkt Heilig-Geist-Haus     Insel Schütt  \
Tag        Uhrzeit                                                       
2022-07-29 17:00:00             NaN               NaN     Gorilla Club   
           19:00:00  Razzzones; The               NaN  Botticelli Baby   
           20:00:00             NaN               NaN              NaN   
           20:30:00             NaN               NaN              NaN   
           21:00:00             NaN               NaN              NaN   

Location            Kreuzigungshof     Lorenzer Platz  \
Tag        Uhrzeit                                      
2022-07-29 17:00:00            NaN                NaN   
           19:00:00            NaN       Hanna Sikasa   
           20:00:00            NaN                NaN   
           20:30:00            NaN  Rooms in Brucklyn   
           21:00:00            NaN                NaN   

Location                                  Pfarrhof St. Sebald  \
Tag        Uhrzeit                                              
2022-07-29 17:00:00                                       NaN   
           19:00:00  Ton Steine Scherben & Gymmick - Gespräch   
           20:00:00                                       NaN   
           20:30:00                                       NaN   
           21:00:00                                       NaN   

Location                Sebalder Platz      St. Katharina  \
Tag        Uhrzeit                                          
2022-07-29 17:00:00                NaN                NaN   
           19:00:00  Fortuna Ehrenfeld  Loxandra Ensemble   
           20:00:00                NaN                NaN   
           20:30:00                NaN                NaN   
           21:00:00                NaN                NaN   

Location                         Strassenbühne            Trödelmarkt  
Tag        Uhrzeit                                                     
2022-07-29 17:00:00                        NaN                    NaN  
           19:00:00  Nürnberger Hauptstadtchor  Johnny & The Yooahoos  
           20:00:00              Die Desirenen                    NaN  
           20:30:00                        NaN                    NaN  
           21:00:00                 Nomi & Mac                    NaN

In [73]:
df_pivoted.to_excel('Bardentreffen_Programm.xlsx')